In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
# !pip3 install tensorflow

In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-07-26 12:02:41.057002: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 12:02:41.102626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 12:02:41.893229: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
model = pickle_load_or_create('../models/convLSTM_128_max_pool', lambda: None, True)
print(model)

None


In [5]:
from DataGeneratorBordersDB import DataGeneratorSkillBorders


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
config = pickle_load_or_create('convLSTM_128_max_pool', lambda:{
    'convolution': (3,3),
    'time_length':8,
    'dim':128,
    'rgb':True,
    'unique_labels': 4,
}, config=True)
config

{'convolution': (3, 3),
 'time_length': 8,
 'dim': 128,
 'rgb': True,
 'unique_labels': 4}

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Dense, Flatten, Input, MaxPooling2D
from tensorflow.keras.optimizers import Adam

if model is None:
    model = Sequential()    
    model.add(ConvLSTM2D(filters=64, kernel_size=config['convolution'],
                         input_shape=(config['time_length'], config['dim'],  config['dim'], 3 if config['rgb'] else 1),
                         padding='same', return_sequences=True))
    model.add(TimeDistributed(MaxPooling2D()))
    model.add(BatchNormalization())
    
    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                         padding='same', return_sequences=True))
    model.add(TimeDistributed(MaxPooling2D()))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=128, kernel_size=(3, 3),
                         padding='same', return_sequences=True))
    model.add(TimeDistributed(MaxPooling2D()))
    model.add(BatchNormalization())
    
    # Apply TimeDistributed Dense layer to each time step
    model.add(TimeDistributed(Flatten()))  # Flatten each frame
    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(TimeDistributed(Dense(config['unique_labels'], activation='softmax')))
    
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
else:
    model = model.model

/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 8, 128, 128,    │       154,624 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 8, 64, 64, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 64, 64, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 8, 64, 64, 64)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 8, 32, 32, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 32, 32, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 8, 32, 32, 128) │       885,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 8, 16, 16, 128) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 16, 16, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 8, 32768)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 8, 128)         │     4,194,432 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 8, 4)           │           516 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,531,012 (21.10 MB)

 Trainable params: 5,530,500 (21.10 MB)

 Non-trainable params: 512 (2.00 KB)

In [9]:
# Parameters
params = {'dim': (config['dim'],config['dim']),
          'time_length': config['time_length'],
          'n_classes': config['unique_labels'],
          'n_channels': 3 if config['rgb'] else 1,
          'shuffle': True,
}

training_generator = DataGeneratorSkillBorders(train=True, **params)
test_generator = DataGeneratorSkillBorders(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        3               187             1            187
1        7               168             2            355
2        1                93             3            448
3        5               318             4            766
4        4               261             5           1027
5        2               180             6           1207
6        6               112             7           1319
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        9                87             1             87
1        8               127             2            214


In [10]:
training_generator.batch_order

,videoID,batches_in_video,random_order,total_batches
0,3,187,1,187
1,7,168,2,355
2,1,93,3,448
3,5,318,4,766
4,4,261,5,1027
5,2,180,6,1207
6,6,112,7,1319


In [11]:
%%time
X, y = training_generator.__getitem__(25)

 getitem, batch_nr 25 train is: True
CPU times: user 1.19 s, sys: 69.3 ms, total: 1.26 s
Wall time: 372 ms


In [12]:
X.shape

(1, 8, 128, 128, 3)

In [13]:
X

array([[[[[0.3254902 , 0.3372549 , 0.3372549 ],
          [0.32156863, 0.33333333, 0.33333333],
          [0.3254902 , 0.3372549 , 0.3372549 ],
          ...,
          [0.28235294, 0.29411765, 0.29411765],
          [0.28627451, 0.29803922, 0.29803922],
          [0.29411765, 0.30588235, 0.30588235]],

         [[0.32941176, 0.34117647, 0.34117647],
          [0.31372549, 0.3254902 , 0.3254902 ],
          [0.31372549, 0.3254902 , 0.3254902 ],
          ...,
          [0.29411765, 0.30588235, 0.30588235],
          [0.29411765, 0.30588235, 0.30588235],
          [0.29411765, 0.30588235, 0.30588235]],

         [[0.33333333, 0.34509804, 0.34509804],
          [0.32941176, 0.34117647, 0.34117647],
          [0.31764706, 0.32941176, 0.32941176],
          ...,
          [0.29411765, 0.30588235, 0.30588235],
          [0.30980392, 0.30980392, 0.30980392],
          [0.29803922, 0.30980392, 0.30980392]],

         ...,

         [[0.74901961, 0.71764706, 0.67058824],
          [0.71372549,

In [14]:
y.shape

(1, 8, 1)

In [15]:
y

array([[[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [3],
        [3]]])

In [16]:
y.shape


(1, 8, 1)

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=2, 
                    validation_data=test_generator, shuffle=False)

len called len is: 1319
len called len is: 1319
 getitem, batch_nr 0 train is: True
fill_time_length_dimension_called
 getitem, batch_nr 1 train is: True
Epoch 1/2
len called len is: 1319
len called len is: 1319
 getitem, batch_nr 0 train is: True
fill_time_length_dimension_called
 getitem, batch_nr 1 train is: True
   1/1319 ━━━━━━━━━━━━━━━━━━━━ 2:50:40 8s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 2 train is: True
   2/1319 ━━━━━━━━━━━━━━━━━━━━ 32:07 1s/step - accuracy: 1.0000 - loss: 3.5763e-07   getitem, batch_nr 3 train is: True
   3/1319 ━━━━━━━━━━━━━━━━━━━━ 32:14 1s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 4 train is: True
   4/1319 ━━━━━━━━━━━━━━━━━━━━ 32:25 1s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 5 train is: True
   5/1319 ━━━━━━━━━━━━━━━━━━━━ 32:47 1s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 6 train is: True
   6/1319 ━━━━━━━━━━━━━━━━━━━━ 34:15 2s/step - accuracy: 1.0000 - loss: 3.5763e-07 getit

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history)

In [ ]:
model.fit?

In [ ]:
with open('../models/frames_skillborder_convLSTM_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)